# Understanding patch-clamp data

In this notebook, we look in detail at processing data obtained from patch-clamp experiments.

In particular, we focus on **manual patch clamp** experiments in the **whole cell configuration**.
These can be used to either

1. pass current through the membrane and measure the resulting voltage (_current clamp_); or 
2. control the membrane voltage and measure the resulting current (_voltage clamp_).

We will focus exclusively on **voltage clamp**.

## The very basics

A detailed descriptions of the patch clamp technique can be found in the chapter [Penner (1995) A Practical Guide to Patch Clamping](https://doi.org/10.1007/978-1-4419-1229-9_1), the (short) book [Molleman (2002) Patch Clamping](https://doi.org/10.1002/0470856521), or the (slightly older) article [Hamill, Marty et al., Sigworth (1981) Improved patch-clamp techniques for high-resolution current recording from cells and cell-free membrane patches](https://doi.org/10.1007/BF00656997).

In brief, a cell is placed in a bath containing an electrode (the _bath electrode_) and an ion-rich solution that can conduct electricity.
A patch clamp _pipette_ is constructed by heating and mechanically pulling a small glass tube, then filling it with another ion-rich solution, and inserting an electrode into the open end.
The sharp end of the pipette is then placed against the cell membrane and suction, electric pulses, or chemicals are used to break the membrane inside the pipette opening, thereby establishing a connection to the inside of the cell.
A _patch clamp amplifier_ is attached to both electrodes, and can now be used to measure and manipulate the voltage between the bath and pipette electrodes.


## Patch-clamp amplifier basics

Now that we have access to the inside & outside of the cell, how can we control its voltage and measure the current?

The answer involves a lot of electronics, but a brief outline is given below.
It follows the exposition in [Sigworth (1995) Electronic design of the patch clamp](https://doi.org/10.1007/978-1-4419-1229-9_4), but also uses [Weerakoon et al., 2009](https://doi.org/10.1109/TBCAS.2008.2005419) and [Lei et al., 2020](https://doi.org/10.1098/rsta.2019.0348).

The image below (adapted from Sigworth 1995) shows a very simple schematic to measure small currents.
A battery $V_c$ is attached to a pipette, and a resistor with a known resistance $R$ is introduced.
To obtain $I$, we measure the voltage drop $V_\text{out} = IR$ across the resistor, and use

$$ I = V_\text{out} / R $$

<img src="resources/patch-amp-1-too-simple.png" />

_**Figure 1**: A very simple current measuring device._

By choosing a very large $R$ we get a measurable $V_\text{out}$ for even very small $I$.

However, it also creates a big difference between $V_c$, which we control, and the voltage over the patch clamp assembly (pipette, cell, bath etc.).

A clever trick with an [op amp](https://en.wikipedia.org/wiki/Operational_amplifier) can get us around this:

<img src="resources/patch-amp-2-with-op-amp.png" />

_**Figure 2**: A better way to measure small currents._

To analyse this circuit, we use two properties of **an idealised** op-amp:

1. Connected in a negative feedback loop like above, the op amp instantaneously adjusts its output $V_o$ until the voltages at its input terminals ($+$ and $-$) are the same.
2. No current flows into (or out of) the input terminals.

From the first property we get $V_- = V_c$, and the second property gives us $I = I_R$.
The voltage-drop across the resistor is then

$$ V_o - V_- = V_o - V_c = I_R R = I R $$

And so if we can measure $V_\text{out} \equiv V_o - V_c$ we can use the known value of $R$ to calculate

$$I = V_\text{out} / R$$

Finally, we add a [difference amplifier](https://en.wikipedia.org/wiki/Differential_amplifier) to $V_o$ and $V_c$.
A difference amplifier takes the voltage between its two inputs and multiplies it by a fixed factor, using an external power source.
Here we use an amplification factor of 1 so that it acts as a simple _buffer_.
This means that the power drawn from $V_{out}$ by any connected measurement equipment will be provided by the amplifier's power source instead of the circuit we're trying to measure.

<img src="resources/patch-amp-3-diff-amp.png" />

_**Figure 2**: A difference amplifier buffers $V_{out}$._

### Diversion 3: Shunt capacitance

The assumptions we made about the op amp are wrong in lots of interesting ways, and in particular when analysing _changing voltages_ instead of the static equilibrium cases we've been considering so far.
An analysis in [Sigworth 1995](https://doi.org/10.1007/978-1-4419-1229-9_4) however, suggests that a more important limitation is the "shunt capacitance" of the resistor $R$ in Figure 2.
To make our measurements precise enough for tiny current currents, we want an resistor with a very big $R$ (10-100GΩ, Sigworth 1995).
But in addition to a resistance, these components introduce a parallel, or ["_shunt_"](https://en.wikipedia.org/wiki/Shunt_(electrical)) capacitance (in the range of 0.1 pF, Sigworth 1995).

With this, the circuit of Figure 2 becomes:

<img src="resources/patch-amp-3c-with-shunt.png" />

We can redo the analysis with this capacitor in place, to see how it affects $V_out$.

Because there are now two pathways for the current $I$ to flow through we write

\begin{align}
I &= I_R + I_C \\
  &= (V_o - V_-) / R + C \frac{d}{dt}\left(V_o - V_-\right) \\
  &= V_\text{out} / R + C \dot{V}_\text{out} \\
V_\text{out} &= I R - R C \dot{V}_\text{out}
\end{align}

To allow for the idea that the current we calculate is no longer equal to $I$, we introduce a new symbol $I_\text{obs} \equiv V_\text{out} / R$.
With a constant $R$, we have $\dot{V}_\text{out} = R \dot{I}_\text{out}$ and

\begin{align}
V_\text{out} &= I R - R C \dot{V}_\text{out} \\
I_\text{out} &= I - R C \dot{I}_\text{out} \\
\dot{I}_\text{out} &= \frac{I - I_\text{out}}{R C} = \frac{I - I_\text{out}}{\tau}
\end{align}

In other words, if $I$ is larger than $I_\text{out}$ then $I_\text{out}$ will grow, but if $I$ is smaller then $I_\text{out}$ will shrink, and so $I_\text{out}$ follows $I$ with a time constant $\tau = RC$.

<img src="resources/patch-amp-3d-phase-diagram.png" />

Section 2.2 in [Sigworth 1995](https://doi.org/10.1007/978-1-4419-1229-9_4) points out that this $\tau$ is in the order of 1 ms, which seriously hampers the ability to voltage clamp.
Luckily, this can be compensated for with clever circuitry, as described in Sigworth's section 2.3.

In the remainder of this notebook we will assume the shunt capacitance has been dealt with, and omit it from the schematics.

**Add some note here about old vs new: Sigworth says Rf 10-100GOhm, Cf 0.1pF for tau in order of ms. Then Weerakoon 2009 (planar amp) says Rf 25MOhm, _deliberately added_ Cf=0.3pF for tau = 7.5 us.**

## Adding the pipette

$V_p$, $C_p$, $R_s$